# 🔗 LangChain + LLM Abstractions

In this notebook we’ll move from the raw OpenAI SDK to **LangChain**, which provides higher-level abstractions to make LLM use easier.

We’ll cover:
1. Setting up `ChatOpenAI`.  
2. Prompt templates & the **LangChain Expression Language (LCEL)**.  
3. Varying parameters like `temperature` and `top_p`.  
4. **Streaming** responses with callbacks.  
5. **Batching & parallel calls** with `.batch()`.  
6. Getting **structured outputs** with Pydantic models.  
7. Swapping configs at runtime (`.bind`, `.with_config`).  
8. (Optional) Using Azure OpenAI through LangChain.

By the end, you’ll see how LangChain **simplifies interaction, chaining, and integration** with LLMs.

In [11]:
import os
from typing import Any

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,   # same knobs as OpenAI
    # top_p=0.9,
    api_key=os.getenv("OPENAI_API_KEY"),
)

llm.invoke("Give me two quick tips for learning Python.")

AIMessage(content="Sure! Here are two quick tips for learning Python:\n\n1. **Practice Regularly**: Consistency is key when learning a programming language. Set aside time each day or week to practice coding. Work on small projects, solve coding challenges on platforms like LeetCode or HackerRank, and try to implement what you learn in real-world scenarios.\n\n2. **Utilize Online Resources**: Take advantage of the wealth of online resources available. Websites like Codecademy, freeCodeCamp, and Coursera offer interactive Python courses. Additionally, the official Python documentation is a great reference for understanding language features and libraries. Engaging with community forums like Stack Overflow can also help you troubleshoot and learn from others' experiences.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 16, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens'

In [2]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, concise assistant."),
    ("user", "Summarize this in 2 bullets:\n\n{text}")
])

In [4]:
text = "Transformers use attention to weigh context; embeddings turn tokens into vectors."

In [ ]:
prompt_rendered = prompt.invoke({"text": text})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful, concise assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Summarize this in 2 bullets:\n\nTransformers use attention to weigh context; embeddings turn tokens into vectors.', additional_kwargs={}, response_metadata={})])

In [7]:
response = llm.invoke(prompt_rendered)

In [9]:
content = StrOutputParser().invoke(response)
print(content)

- Transformers utilize attention mechanisms to prioritize context in processing information.
- Embeddings convert tokens into vector representations for effective data handling.


In [ ]:
chain = prompt | llm | StrOutputParser()
chain.invoke({"text": text})

In [13]:
from langchain_core.callbacks import BaseCallbackHandler

class PrintHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
        print(token, end="")

stream_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, streaming=True, callbacks=[PrintHandler()])
(stream_llm | StrOutputParser()).invoke("Stream a 5-sentence haiku about rain.")
print()

Gentle drops descend,  
Whispers weave through emerald,  
Earth drinks in the sound.  
Clouds embrace the twilight,  
Nature's pulse in rhythm.  


In [14]:
questions = [
    "What is overfitting?",
    "Explain dropout in one line.",
    "Contrast precision vs recall briefly."
]
# Runnable.batch for parallel execution
answers = (llm | StrOutputParser()).batch(questions)
for q, a in zip(questions, answers):
    print(f"Q: {q}\nA: {a}\n")


Q: What is overfitting?
A: Overfitting is a common problem in machine learning and statistical modeling where a model learns not only the underlying patterns in the training data but also the noise and outliers. This results in a model that performs very well on the training data but poorly on unseen data or test data. Essentially, the model becomes too complex and tailored to the specific dataset it was trained on, losing its ability to generalize to new, unseen examples.

Key characteristics of overfitting include:

1. **High Training Accuracy**: The model achieves very high accuracy on the training dataset.
2. **Low Test Accuracy**: The model performs poorly on validation or test datasets, indicating that it cannot generalize well.
3. **Complexity**: Overfitting often occurs with overly complex models that have too many parameters relative to the amount of training data.

To mitigate overfitting, several strategies can be employed:

- **Simplifying the Model**: Using a less complex 

In [15]:
from pydantic import BaseModel, Field

class Flashcard(BaseModel):
    term: str = Field(..., description="Short term")
    definition: str = Field(..., description="One-sentence definition")

structured_llm = llm.with_structured_output(Flashcard)  # Let LC coax JSON→model
card = structured_llm.invoke("Create a flashcard about 'positional encoding' in Transformers.")
card

Flashcard(term='Positional Encoding', definition='A technique used in Transformers to inject information about the position of tokens in a sequence, enabling the model to understand the order of words.')